# 패키지 가져오기

In [1]:
import pandas as pd
import numpy as np
import random
import os

import matplotlib.pyplot as plt
import seaborn as sns

import datetime
from pytz import timezone

import gc

# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import Dataset, DataLoader

# from transformers import AutoTokenizer, AutoModel

from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder
# from sklearn.metrics import accuracy_score, f1_score

from tqdm.notebook import tqdm

In [2]:
# def set_seeds(seed):
#     np.random.seed(seed)
#     random.seed(seed)
#     torch.manual_seed(seed)
#     torch.cuda.manual_seed(seed)
#     torch.backends.cudnn.deterministic = True
#     torch.backends.cudnn.benchmark = False

# set_seeds(777)

# 디렉토리 설정

In [3]:
os.chdir('/content/drive/MyDrive/Competitions/book_recommend')
os.getcwd()

'/content/drive/MyDrive/Competitions/book_recommend'

In [4]:
from modules.preprocess import Preprocessor

# 데이터 전처리

## 가져오기

In [5]:
train_org = pd.read_csv('dataset/train.csv')
submit_org = pd.read_csv('dataset/test.csv')
print(len(train_org))

871393


## 살펴보기

In [6]:
train_org.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 871393 entries, 0 to 871392
Data columns (total 10 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   ID                   871393 non-null  object 
 1   User-ID              871393 non-null  object 
 2   Book-ID              871393 non-null  object 
 3   Book-Rating          871393 non-null  int64  
 4   Age                  871393 non-null  float64
 5   Location             871393 non-null  object 
 6   Book-Title           871393 non-null  object 
 7   Book-Author          871393 non-null  object 
 8   Year-Of-Publication  871393 non-null  float64
 9   Publisher            871393 non-null  object 
dtypes: float64(2), int64(1), object(7)
memory usage: 66.5+ MB


In [7]:
train_org.describe()

,Book-Rating,Age,Year-Of-Publication
count,871393.000000,871393.000000,871393.000000
mean,2.820579,36.799861,1968.903339
std,3.846906,12.065509,228.077752
min,0.000000,0.000000,-1.000000
25%,0.000000,31.000000,1991.000000
50%,0.000000,35.000000,1997.000000
75%,7.000000,41.000000,2001.000000
max,10.000000,244.000000,2021.000000


In [16]:
train_org.sample(10).sort_index()

,ID,User-ID,Book-ID,Book-Rating,Age,Location,Book-Title,Book-Author,Year-Of-Publication,Publisher
143559,TRAIN_143559,USER_15223,BOOK_065519,9,61.0,"lloydminster, saskatchewan, canada",ETHAN FROME,Edith Wharton,1997.0,Scribner
150447,TRAIN_150447,USER_16088,BOOK_222022,0,52.0,"scottsdale, arizona, usa",Bias: A CBS Insider Exposes How the Media Dist...,Bernard Goldberg,2001.0,Regnery Publishing
171807,TRAIN_171807,USER_18482,BOOK_157160,0,25.0,"apo, new york,",Byzantium (Harper Fiction),Stephen R. Lawhead,1997.0,HarperCollins
184034,TRAIN_184034,USER_20042,BOOK_055422,0,31.0,"danbury, connecticut, usa",The Ring (R),Piers Anthony,1991.0,Tor Books
188133,TRAIN_188133,USER_20359,BOOK_266016,0,27.0,"edinburg, texas, usa",Sense of Evil,KAY HOOPER,2003.0,Bantam
317929,TRAIN_317929,USER_33847,BOOK_153992,9,35.0,"santa monica, california, usa",Naked,David Sedaris,1997.0,Little Brown &amp; Co
628044,TRAIN_628044,USER_65669,BOOK_027789,6,35.0,"huntington beach, california, usa",MCSE : TCP/IP Study Guide,Todd Lammle,1997.0,Sybex Inc
674614,TRAIN_674614,USER_70735,BOOK_108350,10,35.0,"carlsbad, california, usa",The Hours: A Novel,Michael Cunningham,2002.0,Picador
736481,TRAIN_736481,USER_76332,BOOK_123825,0,37.0,"fairbanks, alaska, usa",Disciplines of Beautiful Woman,Anne Ortlund,1991.0,W Publishing Group
840271,TRAIN_840271,USER_88455,BOOK_261929,0,35.0,"gainesville, florida, usa",The Firm,John Grisham,1992.0,Bantam Dell Publishing Group


### User

In [17]:
train_org['User-ID'].value_counts()

USER_56601    11143
USER_54845     6456
USER_52453     5814
USER_73501     4290
USER_03411     3453
              ...  
USER_39404        1
USER_39405        1
USER_39406        1
USER_39408        1
USER_92100        1
Name: User-ID, Length: 83256, dtype: int64

In [21]:
train_org.groupby('User-ID')['Book-Rating'].mean()

User-ID
USER_00000    4.750000
USER_00001    4.000000
USER_00002    0.000000
USER_00003    5.954545
USER_00004    6.500000
                ...   
USER_92096    0.000000
USER_92097    0.000000
USER_92098    0.000000
USER_92099    8.000000
USER_92100    0.000000
Name: Book-Rating, Length: 83256, dtype: float64

### Location

In [22]:
train_org.loc[train_org['Location'].apply(lambda x: x.split(',')[-1].strip()) == '']

,ID,User-ID,Book-ID,Book-Rating,Age,Location,Book-Title,Book-Author,Year-Of-Publication,Publisher
158,TRAIN_000158,USER_00035,BOOK_106884,5,35.0,"west vancouver, ,",Healthy Wealthy &amp; Dead (Phoebe Fairfax Mys...,Suzanne North,1995.0,NeWest Press
160,TRAIN_000160,USER_00040,BOOK_074446,0,35.0,"irving, ,",AGENDA,Bob Woodward,1994.0,Simon &amp; Schuster
161,TRAIN_000161,USER_00040,BOOK_239154,0,35.0,"irving, ,",Daughter of Fortune: A Novel,Isabel Allende,2001.0,HarperTorch
460,TRAIN_000460,USER_00102,BOOK_005446,0,36.0,"perry, florida,",The Well-Filled Microwave Cookbook (Well-Fille...,Victoria Wise,1996.0,Workman Publishing
461,TRAIN_000461,USER_00102,BOOK_046686,0,36.0,"perry, florida,",The Body Farm,Patricia Daniels Cornwell,1995.0,Berkley Publishing Group
...,...,...,...,...,...,...,...,...,...,...
870614,TRAIN_870614,USER_92041,BOOK_229545,8,35.0,"golden, colorado,",Miss Julia Speaks Her Mind : A Novel,Ann B. Ross,2000.0,Perennial
870615,TRAIN_870615,USER_92041,BOOK_239149,5,35.0,"golden, colorado,",The Lake House,James Patterson,2003.0,"Little, Brown"
870857,TRAIN_870857,USER_92073,BOOK_112077,6,35.0,"cincinnati, ,",A Night to Remember,WALTER LORD,1997.0,Bantam
870858,TRAIN_870858,USER_92073,BOOK_156207,9,35.0,"cincinnati, ,",Mr. Murder,Dean R. Koontz,1996.0,Berkley Publishing Group


In [19]:
train_org.iloc[50342]['Location'].split(',')

['los gatos', ' ', '']

### Age

In [12]:
# plt.figure(figsize=(18, 5))
# sns.barplot(train_org['Age'].value_counts())
# plt.xticks(rotation=90, fontsize=8)
# plt.show()

In [13]:
train_org['Age'].value_counts()

35.0     251455
33.0      27349
29.0      25456
30.0      23085
31.0      22515
          ...  
118.0         1
109.0         1
147.0         1
123.0         1
132.0         1
Name: Age, Length: 137, dtype: int64

In [14]:
prep = Preprocessor()
tmp = prep.run(train_org)
# tmp['Age'].value_counts()

In [15]:
tmp.describe()

,Book-Rating,Age,Year-Of-Publication
count,871393.000000,871393.000000,871393.000000
mean,2.820579,36.686255,1996.652228
std,3.846906,11.004240,4.386969
min,0.000000,0.000000,1991.000000
25%,0.000000,31.000000,1991.000000
50%,0.000000,35.000000,1997.000000
75%,7.000000,41.000000,2001.000000
max,10.000000,100.000000,2021.000000
